In [0]:
#import moduels
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
%matplotlib inline  
import glob
import tensorflow as tf
import numpy as np
import h5py
import math
import time
from random import shuffle
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics
from keras.optimizers import Adam, SGD
from keras.layers import Flatten
from keras.regularizers import l2
from keras.layers import Input
from keras import models
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers import Dense,add, Dropout, Activation
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from imblearn.over_sampling import SMOTE
!pip install impyute

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df  = pd.read_csv("/content/drive/My Drive/GreatLearning Hackathon /GS_data_V1.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,13,14,51,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
drop_list = df[['maturity_bond', 'duration_bond', '3yrs_treynor_ratio_category','5yrs_treynor_ratio_category','10yrs_treynor_ratio_fund', '10yrs_treynor_ratio_category', 'mmc']]
df.drop(drop_list, axis = 1, inplace = True)

In [0]:
df["ps_ratio"] = df["ps_ratio"].str.replace(",","").astype(float)
df["pc_ratio"] = df["pc_ratio"].str.replace(",","").astype(float)
df["pe_ratio"] = df["pe_ratio"].str.replace(",","").astype(float)
df["3yrs_treynor_ratio_fund"] = df["3yrs_treynor_ratio_fund"].str.replace(",","").astype(float)
df["5yrs_treynor_ratio_fund"] = df["5yrs_treynor_ratio_fund"].str.replace(",","").astype(float)

In [0]:
df.columns

Index(['investment_class', 'total_assets', 'yield', 'greatstone_rating', 'tag',
       'fund_size', 'return_ytd', 'fund_id', 'fund_ratio_net_annual_expense',
       'pb_ratio', 'ps_ratio', 'pc_ratio', 'pe_ratio', 'bb_rating',
       'below_b_rating', 'others_rating', 'b_rating', 'a_rating', 'aaa_rating',
       'aa_rating', 'bbb_rating', 'portfolio_communication_allocation',
       'portfolio_financial_services', 'portfolio_industrials_allocation',
       'portfolio_tech_allocation', 'portfolio_materials_basic_allocation',
       'portfolio_energy_allocation', 'portfolio_consumer_defence_allocation',
       'portfolio_healthcare_allocation', 'portfolio_property_allocation',
       'portfolio_utils_allocation', 'portfolio_cyclical_consumer_allocation',
       'cash_percent_of_portfolio', 'stock_percent_of_portfolio',
       'bond_percentage_of_porfolio', 'portfolio_convertable',
       'portfolio_others', 'portfolio_preferred',
       'category_ratio_net_annual_expense', '1_month_fund_r

In [0]:
df['yield'] = df['yield'].fillna(df.groupby(['investment_class', 'category'])['yield'].transform('mean'))
df['return_ytd'] = df['return_ytd'].fillna(df.groupby(['category'])['return_ytd'].transform('mean'))
df['5_years_return_category'] = df['5_years_return_category'].fillna(df.groupby(['category'])['5_years_return_category'].transform('mean'))
df['10_years_return_category'] = df['10_years_return_category'].fillna(df.groupby(['category'])['10_years_return_category'].transform('mean'))
df['3_years_return_category'] = df['3_years_return_category'].fillna(df.groupby(['category'])['3_years_return_category'].transform('mean'))
df['fund_return_3years'] = df['fund_return_3years'].fillna(df.groupby(['category'])['fund_return_3years'].transform('mean'))
df['5_years_return_fund'] = df['5_years_return_fund'].fillna(df.groupby(['category'])['5_years_return_fund'].transform('mean'))
df['10_years_return_fund'] = df['10_years_return_fund'].fillna(df.groupby(['category'])['10_years_return_fund'].transform('mean'))
df['fundVscat3yr'] = df['fund_return_3years'] - df['3_years_return_category']
df['fundVscat5yr'] = df['5_years_return_fund'] - df['5_years_return_category']
df['fundVscat10yr'] = df['10_years_return_fund'] - df['10_years_return_category']
df['2016_return_fund'] = df['2016_return_fund'].fillna(df.groupby(['category'])['2016_return_fund'].transform('mean'))
df['2017_return_fund'] = df['2017_return_fund'].fillna(df.groupby(['category'])['2017_return_fund'].transform('mean'))
df['2018_return_fund'] = df['2018_return_fund'].fillna(df.groupby(['category'])['2018_return_fund'].transform('mean'))
df['cash_percent_of_portfolio'] = df['cash_percent_of_portfolio'].fillna(df.groupby(['investment_class'])['cash_percent_of_portfolio'].transform('mean'))
df['cash_percent_of_portfolio'] = df['cash_percent_of_portfolio'].fillna(df['cash_percent_of_portfolio'].mean())      
df['portfolio_convertable'] = df['portfolio_convertable'].fillna(df.groupby(['investment_class'])['portfolio_convertable'].transform('mean'))
df['portfolio_convertable'] = df['portfolio_convertable'].fillna(df['portfolio_convertable'].mean())   

df['portfolio_others'] = df['portfolio_others'].fillna(df.groupby(['investment_class'])['portfolio_others'].transform('mean'))
df['portfolio_others'] = df['portfolio_others'].fillna(df['portfolio_others'].mean())      

df['portfolio_preferred'] = df['portfolio_preferred'].fillna(df.groupby(['investment_class'])['portfolio_preferred'].transform('mean'))
df['portfolio_preferred'] = df['portfolio_preferred'].fillna(df['portfolio_preferred'].mean())     

df['3yrs_treynor_ratio_fund'] = df['3yrs_treynor_ratio_fund'].fillna(df.groupby(['category'])['3yrs_treynor_ratio_fund'].transform('mean'))
df['3yrs_treynor_ratio_fund'] = df['3yrs_treynor_ratio_fund'].fillna(df['3yrs_treynor_ratio_fund'].mean())    


In [0]:
df['fundVscat3yr'] = df['fund_return_3years'] - df['3_years_return_category']
df['fundVscat5yr'] = df['5_years_return_fund'] - df['5_years_return_category']
df['fundVscat10yr'] = df['10_years_return_fund'] - df['10_years_return_category']

In [0]:
def fast_mode(df, key_cols, value_col):
    return (df.groupby(key_cols + [value_col]).size() 
              .to_frame('counts').reset_index() 
              .sort_values('counts', ascending=False) 
              .drop_duplicates(subset=key_cols)).drop(columns='counts')

In [0]:
df.loc[df.investment_class.isnull(), 'investment_class'] = df.category.map(fast_mode(df, ['category'], 'investment_class').set_index('category').investment_class)

In [0]:
df.loc[df.fund_size.isnull(), 'fund_size'] = df.category.map(fast_mode(df, ['category'], 'fund_size').set_index('category').fund_size)

In [0]:
df['fund_size'].value_counts()

Large     14568
Medium     6664
Small      3755
Name: fund_size, dtype: int64

In [0]:
df['portfolio_communication_allocation'] = df['portfolio_communication_allocation']*(df['stock_percent_of_portfolio']/100)
df['portfolio_financial_services'] = df['portfolio_financial_services']*(df['stock_percent_of_portfolio']/100)
df['portfolio_industrials_allocation'] = df['portfolio_industrials_allocation']*(df['stock_percent_of_portfolio']/100)
df['portfolio_tech_allocation'] = df['portfolio_tech_allocation']*(df['stock_percent_of_portfolio']/100)
df['portfolio_materials_basic_allocation'] = df['portfolio_materials_basic_allocation']*(df['stock_percent_of_portfolio']/100)
df['portfolio_energy_allocation'] = df['portfolio_energy_allocation']*(df['stock_percent_of_portfolio']/100)
df['portfolio_consumer_defence_allocation'] = df['portfolio_consumer_defence_allocation']*(df['stock_percent_of_portfolio']/100)
df['portfolio_healthcare_allocation'] = df['portfolio_healthcare_allocation']*(df['stock_percent_of_portfolio']/100)
df['portfolio_property_allocation'] = df['portfolio_property_allocation']*(df['stock_percent_of_portfolio']/100)
df['portfolio_utils_allocation'] = df['portfolio_utils_allocation']*(df['stock_percent_of_portfolio']/100)
df['portfolio_cyclical_consumer_allocation'] = df['portfolio_cyclical_consumer_allocation']*(df['stock_percent_of_portfolio']/100)
df['bb_rating'] = df['bb_rating']*(df['bond_percentage_of_porfolio']/100)
df['below_b_rating'] = df['below_b_rating']*(df['bond_percentage_of_porfolio']/100)
df['others_rating'] = df['others_rating']*(df['bond_percentage_of_porfolio']/100)
df['b_rating'] = df['b_rating']*(df['bond_percentage_of_porfolio']/100)
df['a_rating'] = df['a_rating']*(df['bond_percentage_of_porfolio']/100)
df['aaa_rating'] = df['aaa_rating']*(df['bond_percentage_of_porfolio']/100)
df['bbb_rating'] = df['bbb_rating']*(df['bond_percentage_of_porfolio']/100)
df['aa_rating'] = df['aa_rating']*(df['bond_percentage_of_porfolio']/100)


In [0]:
max_list = ['bb_rating',
       'below_b_rating', 'others_rating', 'b_rating', 'a_rating', 'aaa_rating',
       'aa_rating', 'bbb_rating', 'portfolio_communication_allocation',
       'portfolio_financial_services', 'portfolio_industrials_allocation',
       'portfolio_tech_allocation', 'portfolio_materials_basic_allocation',
       'portfolio_energy_allocation', 'portfolio_consumer_defence_allocation',
       'portfolio_healthcare_allocation', 'portfolio_property_allocation',
       'portfolio_utils_allocation', 'portfolio_cyclical_consumer_allocation',
       'cash_percent_of_portfolio' ]
df['max_alloc'] = df[max_list].idxmax(axis=1)


In [0]:
df.columns

Index(['investment_class', 'total_assets', 'yield', 'greatstone_rating', 'tag',
       'fund_size', 'return_ytd', 'fund_id', 'fund_ratio_net_annual_expense',
       'pb_ratio', 'ps_ratio', 'pc_ratio', 'pe_ratio', 'bb_rating',
       'below_b_rating', 'others_rating', 'b_rating', 'a_rating', 'aaa_rating',
       'aa_rating', 'bbb_rating', 'portfolio_communication_allocation',
       'portfolio_financial_services', 'portfolio_industrials_allocation',
       'portfolio_tech_allocation', 'portfolio_materials_basic_allocation',
       'portfolio_energy_allocation', 'portfolio_consumer_defence_allocation',
       'portfolio_healthcare_allocation', 'portfolio_property_allocation',
       'portfolio_utils_allocation', 'portfolio_cyclical_consumer_allocation',
       'cash_percent_of_portfolio', 'stock_percent_of_portfolio',
       'bond_percentage_of_porfolio', 'portfolio_convertable',
       'portfolio_others', 'portfolio_preferred',
       'category_ratio_net_annual_expense', '1_month_fund_r

In [0]:
samtrain = df[['return_ytd', '2017_return_fund', '2018_return_fund', '5_years_return_fund', 'fund_return_3years', 'fundVscat3yr', 'fundVscat5yr']]

In [0]:
#for col in samtrain.columns:
 #   percentiles = samtrain[col].quantile([0.05,0.95]).values
  #  samtrain[col][samtrain[col] <= percentiles[0]] = percentiles[0]
   # samtrain[col][samtrain[col] >= percentiles[1]] = percentiles[1]

In [0]:
from sklearn.preprocessing import RobustScaler

In [0]:
scaler = RobustScaler()


In [0]:
samtrain = pd.DataFrame(scaler.fit_transform(samtrain), columns = samtrain.columns)

In [0]:
#samtrain = samtrain.round(4)

In [0]:
#from impyute.imputation.cs import mice
#imputed_training=mice(samtrain.values )
#imputed_df = pd.DataFrame(imputed_training, columns = samtrain.columns)
#imputed_df.shape
#imputed_df.to_csv("/content/drive/My Drive/GreatLearning Hackathon /mice_imputed_asset.csv")

In [0]:
#imputed_df = pd.read_csv("/content/drive/My Drive/GreatLearning Hackathon /mice_imputed_asset.csv", index_col=0)

In [0]:
df1 = df[[ 'tag', 'greatstone_rating', 'fund_id','max_alloc']] #, 'investment_class', 'fund_size', 

In [0]:
train_df = pd.concat([df1,samtrain], axis=1)

In [0]:
train_df.shape

(25000, 11)

In [0]:
train_df = pd.get_dummies(train_df, columns = [ 'max_alloc'], drop_first= True) #, 'investment_class', 'fund_size',
key_df = df[['tag','fund_id']]

In [0]:
labels = df[['greatstone_rating']]
submission = train_df[train_df['greatstone_rating'].isnull()]
sub_key = submission[['tag','fund_id']]
submission.drop(['tag','fund_id'], axis =1, inplace = True)
train = train_df.dropna(subset=['greatstone_rating'])

train.drop(['tag','fund_id'], axis =1, inplace = True)
train = train.dropna()
train_data = train.drop('greatstone_rating', axis = 1)
#train_data2 = train_data[['2015_return_fund', '2017_return_fund', '2018_return_fund', 'fund_return_3years', '5_years_return_fund', '10_years_return_fund', 'fundVscat3yr', 'fundVscat5yr', 'fundVscat10yr']]
train_label = train['greatstone_rating']
X_sub = submission.drop('greatstone_rating', axis = 1)
y_sub = submission['greatstone_rating']

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
def traintestsplit(X,Y):
    skf = StratifiedKFold(n_splits=32, random_state=12)
    skf.get_n_splits(X, Y)
    for train_index, test_index in skf.split(X, Y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    return(X_train,X_test,y_train,y_test)

In [0]:
#X_train,X_test,y_train,y_test = traintestsplit(train_data,train_label)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data,train_label, test_size=0.15, random_state=1)

In [0]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((15210, 26), (15210,), (2685, 26), (2685,))

In [0]:
X_train.columns

Index(['return_ytd', '2017_return_fund', '2018_return_fund',
       '5_years_return_fund', 'fund_return_3years', 'fundVscat3yr',
       'fundVscat5yr'],
      dtype='object')

In [0]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
xgb=XGBClassifier()
from sklearn.feature_selection import SelectFromModel
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
import xgboost as xgb

In [0]:
# for grid search
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
gb = GradientBoostingClassifier()
param_grid = {
    'n_estimators': [150,200,250],
    'max_depth': range(3, 5),
    'max_leaf_nodes': range(10, 16)
}

gb_cv = GridSearchCV(estimator=gb,
                     param_grid=param_grid,
                     cv=5)
gb_cv.fit(X_train, y_train)
print(gb_cv.best_params_)

KeyboardInterrupt: ignored

In [0]:
gb = GradientBoostingClassifier(max_depth = 4, max_leaf_nodes = 20,  n_estimators = 200)
gb.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=4,
                           max_features=None, max_leaf_nodes=20,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
y_gb_pred = gb.predict(X_test)
print(f"Gradient Boosting Accuracy: {accuracy_score(y_gb_pred, y_test):.3f}")

Gradient Boosting Accuracy: 0.714


In [0]:
test_pred = gb.predict(X_test)
#cf_matrix = confusion_matrix(y_test, test_pred.round())
#cf_matrix
confusion_matrix(y_test, test_pred)

array([[ 99,   0,   2,   1,   1,   0],
       [  1, 111,  91,   3,   1,   0],
       [  1,  31, 422, 151,   2,   1],
       [  0,   2, 114, 670, 146,   6],
       [  0,   0,   6, 135, 412,  45],
       [  0,   0,   0,   3,  93, 135]])

In [0]:
subpred = gb.predict(X_sub)
sub = pd.DataFrame(subpred)

In [0]:
sub_key['greatstone_rating'] = subpred
predictions = sub_key[['fund_id', 'greatstone_rating']]
predictions.head()
sample_submission = pd.read_csv("/content/drive/My Drive/GreatLearning Hackathon /sample_submission.csv")
sam = pd.concat([sample_submission,predictions]).drop_duplicates(['fund_id'],keep='last')
sam.to_csv("sample_submission_results_XGboosttry_v2.1.csv")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators = 250)
ada.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=250, random_state=None)

In [0]:
y_gb_pred = ada.predict(X_test)
print(f"Gradient Boosting Accuracy: {accuracy_score(y_gb_pred, y_test):.3f}")

Gradient Boosting Accuracy: 0.475


In [0]:
test_pred = ada.predict(X_test)
#cf_matrix = confusion_matrix(y_test, test_pred.round())
#cf_matrix
confusion_matrix(y_test, test_pred)

array([[ 16,  23,   2,   1,   0,   0],
       [  0,  24,  15,   2,   1,   1],
       [  0,  51,  53,  26,   2,   1],
       [  1,   5,  32, 128,  31,  15],
       [  2,   0,   4,  44,  41,  53],
       [  0,   1,   0,   3,  12,  35]])

In [0]:
subpred = ada.predict(X_sub)
sub = pd.DataFrame(subpred)

In [0]:
sub_key['greatstone_rating'] = subpred
predictions = sub_key[['fund_id', 'greatstone_rating']]
predictions.head()
sample_submission = pd.read_csv("/content/drive/My Drive/GreatLearning Hackathon /sample_submission.csv")
sam = pd.concat([sample_submission,predictions]).drop_duplicates(['fund_id'],keep='last')
sam.to_csv("sample_submission_ada_v1.csv")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
from keras.utils import to_categorical
y_train2 = to_categorical(y_train)
y_test2 = to_categorical(y_test)

In [0]:
#@title Default title text
def create_model(): 
  model = Sequential()
  model.add(Dense(1026, input_dim=X_train.shape[1], activation= 'relu'))
  model.add(Dropout(.10))
  model.add(Dense(1026, activation= 'relu'))
  model.add(Dropout(.10))
  model.add(Dense(1026, activation= 'relu'))
  model.add(Dropout(.10))
  model.add(Dense(1026, activation= 'relu'))
  model.add(Dropout(.10))
  model.add(Dense(1026, activation= 'relu'))
  model.add(Dropout(.20))
  model.add(Dense(6, activation= 'softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return(model)

In [0]:
early_stopping = EarlyStopping(monitor='val_acc', patience=25, mode='auto', verbose=0,)
mcp_save = ModelCheckpoint('best_V1_3.h5', verbose=1, monitor='val_accuracy',save_best_only=True)
callback_list = [ mcp_save, early_stopping]

In [0]:
model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 1026)              33858     
_________________________________________________________________
dropout_6 (Dropout)          (None, 1026)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 1026)              1053702   
_________________________________________________________________
dropout_7 (Dropout)          (None, 1026)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1026)              1053702   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1026)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1026)             

In [0]:
#model.load_weights("/content/best_V1_2.h5")

In [0]:
model.fit(
        X_train, y_train2,
        epochs=300,
        validation_data=(X_test, y_test2),
        verbose = 1,
        callbacks= callback_list)

Train on 19375 samples, validate on 625 samples
Epoch 1/300
19375/19375 [==============================] - 43s 2ms/step - loss: 0.9810 - accuracy: 0.5806 - val_loss: 0.8496 - val_accuracy: 0.6640

Epoch 00001: val_accuracy improved from -inf to 0.66400, saving model to best_V1_3.h5
Epoch 2/300
   64/19375 [..............................] - ETA: 43s - loss: 1.0600 - accuracy: 0.5625

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


19375/19375 [==============================] - 43s 2ms/step - loss: 0.8585 - accuracy: 0.6328 - val_loss: 0.7961 - val_accuracy: 0.6480

Epoch 00002: val_accuracy did not improve from 0.66400
Epoch 3/300
19375/19375 [==============================] - 43s 2ms/step - loss: 0.8210 - accuracy: 0.6505 - val_loss: 0.7206 - val_accuracy: 0.6624

Epoch 00003: val_accuracy did not improve from 0.66400
Epoch 4/300
18624/19375 [===========================>..] - ETA: 1s - loss: 0.7857 - accuracy: 0.6622

KeyboardInterrupt: ignored

In [0]:
from keras.models import load_model
model = load_model('/content/best_V1_4.h5')

In [0]:
test_pred = model.predict(X_test)
#cf_matrix = confusion_matrix(y_test, test_pred.round())
#cf_matrix
confusion_matrix(y_test2.argmax(axis =1), test_pred.argmax(axis =1))

In [0]:
subpred = model.predict(X_sub)

In [0]:
sub = pd.DataFrame(subpred.argmax(axis = 1))

In [0]:
#sub.to_csv('/content/drive/My Drive/GreatLearning Hackathon /submission_v4.csv')

In [0]:
from sklearn.metrics import average_precision_score
average_precision_score(y_test2, test_pred,  average='weighted')

Delete 

In [0]:
sub_key['greatstone_rating'] = subpred.argmax(axis = 1)
predictions = sub_key[['fund_id', 'greatstone_rating']]
predictions.head()
sample_submission = pd.read_csv("/content/drive/My Drive/GreatLearning Hackathon /sample_submission.csv")
sam = pd.concat([sample_submission,predictions]).drop_duplicates(['fund_id'],keep='last')
sam.to_csv("sample_submission_results_Neural_V5.csv")